## Load and Publish Model

In this `.ipynb` you will load up the model you created and saved in [`create-save-model.ipynb`](../ml-create-model/create-save-model.ipynb) and publish it to a Kafka topic.

#### Pre-reqs

In the `create-save-model.ipynb` file you installed most of the pre-reqs. For the code in this file you only need a couple of things:

* `confluent-kafka`: the Python package for Kafka.
* Kafka: well, since we are using Kafka - you need it.

#### Kafka

You can either run Kafka on-prem or using Confluent Cloud. Here I am running Kafka using Docker. You can find the `docker-compose.yml` file I use [here](../../docker/docker-compose.yml).

When you have a running Kafka cluster, (whether Docker, on-prem, or Confluent Cloud) you need two topics:

* `models`: this is where we push the model to (and we use in this file).
* `adclicks`: this topic is where we emulate rel-time events (adclicks) is published to.

To create the topics in a Docker environment:

```
docker-compose exec broker kafka-topics --create --bootstrap-server \
localhost:9092 --replication-factor 1 --partitions 1 --topic models

docker-compose exec broker kafka-topics --create --bootstrap-server \
localhost:9092 --replication-factor 1 --partitions 1 --topic adclicks
```

The `\` in the code above denotes a line continuation.

We are almost ready. We need to import the necessary packages.

In [12]:
import numpy as np
import pickle
import json
import socket

### Load the Model

Let us load the model we saved off to [`adcklick.pkl`](../../models/adclick.pkl)

In [13]:
# load in the model
lr_model = pickle.load(open('../../models/adclick.pkl', 'rb'))

# create a bytearray and decode into a string to represent the model
modBin = pickle.dumps(lr_model).decode('latin1')

### Create and Publish the Event

Having the string representation we can create an event. The event looks like so:

`{modelId: int, modelName: string, model: string}`

The `model` field holds the model string.

In [14]:
# create an event
modelEvent = {
    "modelId": 2,
    "modelName": 'adclicks-logreg',
    "model": modBin
}

modelEventString = json.dumps(modelEvent)

#### Setup the Publisher

Let us setup the publisher.

In [15]:
# load in the Producer from the confluent_kafka library
from confluent_kafka import Producer

# create the publisher configuration
conf = {'bootstrap.servers': 'localhost:9092',
        'client.id': socket.gethostname()}

# create the producer
producer = Producer(conf)  


#### Publish the Event

We are now in a position where we can publish the event.

In [ ]:
# we start with setting up a callback function to handle the delivery report
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: {0}: {1}".format(msg.value(), err.str()))
    else:
        print("Message produced: {0}".format(msg.value()))

# publish the model
producer.produce('model', key="1", value=modelEventString, callback=acked)